In [1]:
profs = [
    "Yousra Aafer",
    "Samer Al-Kiswany",
    "N. Asokan",
    "Sepehr Assadi",
    "Joanne Atlee",
    "Gladimir Baranoski",
    "Diogo Barradas",
    "Christopher Batty",
    "Shai Ben-David",
    "Shalev Ben-David",
    "Daniel Berry",
    "Therese Biedl",
    "Eric Blais",
    "Raouf Boutaba",
    "Yuri Boykov",
    "Tim Brecht",
    "Dan Brown",
    "Trevor Brown",
    "Peter Buhr",
    "Wenhu Chen",
    "Charles Clarke",
    "Richard Cleve",
    "Robin Cohen",
    "Gordon Cormack",
    "Khuzaima Daudjee",
    "Nancy Day",
    "Yuntian Deng",
    "Kimon Fountoulakis",
    "Mark Giesbrecht",
    "Michael Godfrey",
    "Ian Goldberg",
    "Sergey Gorbunov",
    "Maura R. Grossman",
    "Toshiya Hachisuka",
    "Mohammad Hajiabadi",
    "Jason Hartford",
    "Xi He",
    "Urs Hengartner",
    "Jesse Hoey",
    "Xiao Hu",
    "Ihab F. Ilyas",
    "Gautam Kamath",
    "Craig S. Kaplan",
    "Lila Kari",
    "Martin Karsten",
    "Florian Kerschbaum",
    "George Labahn",
    "Kate Larson",
    "Lap Chi Lau",
    "Edith Law",
    "Ondřej Lhoták",
    "Ming Li",
    "Yuying Li",
    "Noura Limam",
    "Jimmy Lin",
    "Sihang Liu",
    "Yang Lu",
    "Bin Ma",
    "Sujaya Maiyya",
    "Richard Mann",
    "Stephen Mann",
    "Ali José Mashtizadeh",
    "Shane McIntosh",
    "Meiyappan (Mei) Nagappan",
    "Pengyu Nie",
    "Naomi Nishimura",
    "Rafael Oliveira",
    "Jeff Orchard",
    "Tamer Özsu",
    "Pascal Poupart",
    "Prabhakar Ragde",
    "Mohammad Salahuddin",
    "Ken Salem",
    "Semih Salihoğlu",
    "Éric Schost",
    "Jeffrey Shallit",
    "Freda Shi",
    "Shlomi Steinberg",
    "Arne Storjohann",
    "Chengnian Sun",
    "Mina Tahmasbi Arashloo",
    "David Toman",
    "Richard Trefler",
    "Olga Veksler",
    "Daniel Vogel",
    "Justin Wan",
    "Stephen Watt",
    "Grant Weddell",
    "Bernard Wong",
    "Meng Xu",
    "Yaoliang Yu",
    "Hong Zhang",
    "Hongyang Zhang",
    "Yizhou Zhang",
    "Jian Zhao",
    "Victor Zhong"
]

In [3]:
import requests

#write two clumn csv file
with open('profs1.csv', 'w') as f:
    for prof in profs:
        res = requests.get('https://api.semanticscholar.org/graph/v1/author/search?query=' + prof.replace(' ', '+').lower() + '&fields=name,aliases,affiliations').json()
        if res['total'] == 1:
            f.write(prof + ',' + res['data'][0]['authorId'] + '\n')
            continue
        aff = False
        for prof_d in res['data']:
            if "University of Waterloo" in prof_d['affiliations']:
                f.write(prof + ',' + prof_d['authorId'] + '\n')
                print(prof + ',' + prof_d['authorId'])
                aff = True
                break
        if not aff:
            f.write(prof + ',\n')

Christopher Batty,144366758
Wenhu Chen,2928777
Tamer Özsu,1705151
Hongyang Zhang,40975176


In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

/Users/onurerenarpaci/Desktop/citeQ/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'LABEL_1'

In [26]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'

In [6]:
with open("citations_annotated.csv", "r") as f:
    citations = f.readlines()
    citations = [c.strip().split(",") for c in citations]

In [2]:
with open("llm_annotations_gpt4_3class.csv", "r") as f:
    llm_annotations = f.readlines()
    llm_annotations = [c.strip().split(",") for c in llm_annotations]
    llm_annotations = [[c[0], int(c[1]), int(c[2])] for c in llm_annotations]

# calculate precision, and recall for each class
tp = [0, 0, 0, 0]
fp = [0, 0, 0, 0]
fn = [0, 0, 0, 0]

for annotation in llm_annotations:
    if annotation[1] == annotation[2]:
        tp[annotation[1]] += 1
    else:
        fp[annotation[2]] += 1
        fn[annotation[1]] += 1

precision = [0, 0, 0, 0]
recall = [0, 0, 0, 0]
for i in range(4):
    precision[i] = tp[i] / (tp[i] + fp[i]) if tp[i] + fp[i] != 0 else 0
    recall[i] = tp[i] / (tp[i] + fn[i])

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Precision: [0.875, 0.7142857142857143, 0.4305555555555556, 0.9230769230769231]
Recall: [0.1891891891891892, 0.625, 0.96875, 0.5217391304347826]


In [ ]:
LlmClassifier.get_sentiment_class()

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

chat = [
  {"role": "user", "content": "Hello, how are you?"},
]

In [6]:
tokenizer.apply_chat_template(chat, tokenize=False)

'<s>[INST] Hello, how are you? [/INST]'

In [7]:
with open("citations_annotated.csv", "r") as f:
    citations = f.readlines()
    citations = [c.strip().split(",") for c in citations]

class_count = [0, 0, 0, 0]
for citation in citations:
    class_count[int(citation[1])] += 1

class_count

[37, 8, 32, 23]

In [8]:
precision = [x/100 for x in class_count]
precision

[0.37, 0.08, 0.32, 0.23]

In [3]:
a = ['asd', 'sdklsjei', 'iejirgr', 'skefekl']
a = sorted(a)
a

['asd', 'iejirgr', 'sdklsjei', 'skefekl']